In [3]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
from rasterio.enums import Resampling

def resample_raster(input_raster_path, output_raster_path, new_cell_size=250):
    """
    Resample a raster to a new resolution using the mean of the original cells.

    Parameters:
    - input_raster_path: Path to the input raster file.
    - output_raster_path: Path to output the resampled raster file.
    - new_cell_size: The cell size for the resampled raster, in the same units as the input raster's CRS.
    """
    with rasterio.open(input_raster_path) as dataset:
        # Calculate the new dimensions
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * dataset.res[0] / new_cell_size),
                int(dataset.width * dataset.res[1] / new_cell_size)
            ),
            resampling=Resampling.average
        )

        # Adjust the new transform to the new cell size
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )

        # Define the new metadata
        new_meta = dataset.meta.copy()
        new_meta.update({
            "driver": "GTiff",
            "height": data.shape[1],
            "width": data.shape[2],
            "transform": transform
        })

        # Write the resampled data to the new file
        with rasterio.open(output_raster_path, 'w', **new_meta) as dest:
            dest.write(data)

    print(f'Resampling completed for {output_raster_path}.')

def create_recharge_raster(recharge_hru, value_column, output_raster_path, cell_size=30):
    """
    Create a raster from a specified value column in a GeoDataFrame.

    Parameters:
    - recharge_hru: GeoDataFrame containing the recharge data and geometries.
    - value_column: The column in GeoDataFrame to rasterize.
    - output_raster_path: Path to output the resulting raster file.
    - cell_size: The cell size for the raster, in the same units as the GeoDataFrame's CRS.
    """
    
    # Select only the necessary columns and drop duplicates
    recharge_hru = recharge_hru[['geometry', value_column]].drop_duplicates()
    # Calculate bounds and create the transform
    bounds = recharge_hru.total_bounds
    width = int((bounds[2] - bounds[0]) / cell_size)
    height = int((bounds[3] - bounds[1]) / cell_size)
    transform = from_origin(bounds[0], bounds[3], cell_size, cell_size)

    # Define the output raster's metadata
    meta = {
        'driver': 'GTiff',
        'height': height,
        'width': width,
        'count': 1,
        'dtype': 'float32',
        'crs': recharge_hru.crs,
                    'transform': transform
                }

    # Perform the rasterization and write the output raster
    with rasterio.open(output_raster_path, 'w', **meta) as out_raster:
        out_image = rasterize(
            ((geom, value) for geom, value in zip(recharge_hru.geometry, recharge_hru[value_column])),
            out_shape=(height, width),
            transform=transform,
            fill=np.nan,
            all_touched=True,
            dtype='float32'
        )
        out_raster.write(out_image, 1)

    print(f'Conversion to raster completed for {output_raster_path}.')

# Paths
base_dir = "/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/"
hru_name = "hrus2.shp"
recharge_name = "monthly_recharge_hru.pk1"

# Process each HUC
NAMES = os.listdir(base_dir)
NAMES.remove('log.txt')
for NAME in NAMES:
    output_dir = f"/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/{NAME}/SWAT_MODEL/SWAT_outputs/"
    print(f'creating {output_dir}')
    
    os.makedirs(output_dir, exist_ok=True)
    hru_path = os.path.join(base_dir, NAME, "SWAT_MODEL/Watershed/Shapes", hru_name)
    recharge_path = os.path.join(base_dir, NAME, recharge_name)
    output_raster_MAA = os.path.join(output_dir,f"{NAME}_mean_average_annual_recharge.tif")
    os.remove(output_raster_MAA)
    
    # Skip if files don't exist
    if not os.path.exists(hru_path):
        print(f'{hru_path} not exists')
        continue
    if not os.path.exists(recharge_path):
        continue
    if os.path.exists(output_raster_MAA):
        continue
        
    # Read the data
    hru = gpd.read_file(hru_path)
    recharge = pd.read_pickle(recharge_path)
    recharge['id'] = recharge['id'].astype('int64')
    hru = hru[hru['HRUS'] != 'NA']
    hru['HRUS'] = hru['HRUS'].astype('int64')

    # Merge and calculate mean annual recharge
    recharge_hru = pd.merge(recharge, hru, left_on='id', right_on='HRUS')
    recharge_hru = gpd.GeoDataFrame(recharge_hru, geometry='geometry')
    recharge_hru['mean_average_monthly_perc'] = recharge_hru.groupby(['id','mon'])['perc'].transform('mean')
    recharge_hru['mean_average_annual_perc'] = recharge_hru.groupby(['id', 'yr'])['perc'].transform('mean')
    create_recharge_raster(recharge_hru, 'mean_average_annual_perc', output_raster_MAA, cell_size=30)
    output_raster_resampled = os.path.join(output_dir,f"{NAME}_mean_average_annual_recharge_250m.tif")
    resample_raster (output_raster_MAA, output_raster_resampled, new_cell_size=250)
    
    
    

creating /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500010102/SWAT_MODEL/SWAT_outputs/
Conversion to raster completed for /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500010102/SWAT_MODEL/SWAT_outputs/40500010102_mean_average_annual_recharge.tif.
Resampling completed for /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500010102/SWAT_MODEL/SWAT_outputs/40500010102_mean_average_annual_recharge_250m.tif.
creating /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500010206/SWAT_MODEL/SWAT_outputs/
Conversion to raster completed for /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500010206/SWAT_MODEL/SWAT_outputs/40500010206_mean_average_annual_recharge.tif.
Resampling completed for /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500010206/SWAT_MODEL/SWAT_outputs/40500010206_mean_average_annual_recharge_250m.tif.
creating /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500012304/SWAT_MODEL/SWAT_outputs/
Conve

FileNotFoundError: [WinError 2] The system cannot find the file specified: '/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500070211/SWAT_MODEL/SWAT_outputs/40500070211_mean_average_annual_recharge.tif'

In [4]:
import arcpy
import os

# Set the name of the HUC and the base directory
huc_name = "40500010102"
base_dir = "/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/"

# Construct the output directory and set the workspace
output_dir = os.path.join(base_dir, huc_name, "SWAT_MODEL/SWAT_outputs/")
arcpy.env.workspace = base_dir
arcpy.env.overwriteOutput = True

# Define the path to the input feature class and the output raster
feature_class_path = os.path.join(output_dir, f"{huc_name}_recharge.shp")
output_raster = os.path.join(output_dir,f"{huc_name}_recharge.tif")

# Print paths for verification
print(f"Feature class path: {feature_class_path}")
print(f"Output raster path: {output_raster}")

# Perform the Polygon to Raster conversion
try:
    print('Begin conversion...')
    arcpy.PolygonToRaster_conversion(feature_class_path, 'perc', output_raster, cell_size=50)
    print('Conversion completed successfully.')
except Exception as e:
    print("Error occurred during Polygon to Raster conversion.")
    # Get the geoprocessing messages
    msgs = arcpy.GetMessages(2)
    print("ArcPy Messages:")
    print(msgs)

# Check if the output raster was created
if os.path.exists(output_raster):
    print(f"Output raster was created: {output_raster}")
else:
    print(f"Output raster was not created: {output_raster}")


Feature class path: /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500010102\SWAT_MODEL/SWAT_outputs/40500010102_recharge.shp
Output raster path: 40500010102_recharge.tif
Begin conversion...
Error occurred during Polygon to Raster conversion.
ArcPy Messages:
ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (PolygonToRaster).

Output raster was not created: 40500010102_recharge.tif


In [9]:
import geopandas as gpd
gpd.read_file(feature_class_path)['perc']

0        19.533
1        19.533
2        19.533
3        19.533
4        19.533
          ...  
13173     5.118
13174     5.143
13175    44.075
13176     0.000
13177     0.000
Name: perc, Length: 13178, dtype: float64

Feature class path: /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500010102\SWAT_MODEL/SWAT_outputs/40500010102_recharge.shp
Output raster path: /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500010102\SWAT_MODEL/SWAT_outputs/40500010102_recharge.tif
Conversion completed successfully.
Output raster was created: /data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/40500010102\SWAT_MODEL/SWAT_outputs/40500010102_recharge.tif


In [7]:
gpd.read_file(feature_class_path)

,perc,geometry
0,19.533,"POLYGON ((3961574.061 51072.996, 3961547.804 5..."
1,19.533,"POLYGON ((3961574.061 51020.482, 3961574.061 5..."
2,19.533,"POLYGON ((3961521.547 51020.482, 3961469.033 5..."
3,19.533,"POLYGON ((3961442.776 50994.225, 3961442.776 5..."
4,19.533,"POLYGON ((3961469.033 50967.969, 3961469.033 5..."
...,...,...
13173,5.118,"POLYGON ((3963464.567 36920.610, 3963464.567 3..."
13174,5.143,"POLYGON ((3963464.567 36894.353, 3963464.567 3..."
13175,44.075,"POLYGON ((3963464.567 36868.097, 3963464.567 3..."
13176,0.000,"POLYGON ((3964015.964 37865.853, 3964015.964 3..."


In [5]:
load_raster_args = {
    'lists_of_huc': lists_of_huc,
    'extract_method': extract_method,
    'RESOLUTION': RESOLUTION,
    'NAME': NAME}

def generate_shapefile_paths(NAME):
    return {
        
        "lakes"    : f'SWATplus_by_VPUID/huc12/{NAME}/SWAT_MODEL/Watershed/Shapes/SWAT_plus_lakes.shp',
        "rivers"   : f'SWATplus_by_VPUID/huc12/{NAME}/SWAT_MODEL/Watershed/Shapes/rivs1.shp',
        "grids"    : f'/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/{NAME}/MODFLOW_250/Grids_MODFLOW.pk1',
        "recharge" : f'/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/{NAME}/SWAT_MODEL/SWAT_outputs/{NAME}_recharge.shp',



    }
def rasterize_SWAT_features(feature_type, output_raster_path, ref_raster_path, load_raster_args):
    
    lists_of_huc = load_raster_args['lists_of_huc']
    extract_method = load_raster_args['extract_method']
    RESOLUTION = load_raster_args['RESOLUTION']
    NAME = load_raster_args['NAME']
    shapefile_paths = generate_shapefile_paths(NAME)
    # Define paths based on the type of feature
    if feature_type == "lakes":
        
        feature_path = shapefile_paths ["lakes"]
        temp_feature_path =  f'SWATplus_by_VPUID/huc12/{NAME}/SWAT_MODEL/Watershed/Shapes/lakes1_modflow.shp'
        print(f'SWATplus_by_VPUID/huc12/{NAME}/SWAT_MODEL/Watershed/Shapes/lakes1_modflow.shp')
        #print(arcpy.Describe(temp_feature_path).shapeType)
        
    if feature_type == "recharge":
        
        feature_path = shapefile_paths ["recharge"]
        temp_feature_path =  f'SWATplus_by_VPUID/huc12/{NAME}/SWAT_MODEL/SWAT_outputs/recharge_modflow.shp'
        print(f'SWATplus_by_VPUID/huc12/{NAME}/SWAT_MODEL/Watershed/Shapes/recharge_modflow.shp')
        #print(arcpy.Describe(temp_feature_path).shapeType)

    elif feature_type == "rivers":
        
        feature_path = shapefile_paths ["rivers"]
        temp_feature_path = f'SWATplus_by_VPUID/huc12/{NAME}/SWAT_MODEL/Watershed/Shapes/rivs1_modflow.shp'
    
    else:
        print(f"Unknown feature type: {feature_type}. Supported types are 'lakes' and 'rivers'.")
        return

    modflow_grid_path = shapefile_paths ['grids']
    
    # Set overwrite to true for arcpy    
    # Read the shapefile using geopandas
    
    original_feature = gpd.read_file(feature_path)
    
    if os.path.exists(modflow_grid_path):
        
        modflow_grid = gpd.GeoDataFrame(pd.read_pickle(modflow_grid_path))
        modflow_feature_grid = original_feature.overlay(modflow_grid, how='intersection')

        if feature_type == "lakes":
            modflow_feature_grid['lake_area'] = modflow_feature_grid.geometry.area
            modflow_feature_grid['COND'] = modflow_feature_grid['lake_area']
        elif feature_type == "rivers":
            modflow_feature_grid['Len3'] = modflow_feature_grid.geometry.length
            modflow_feature_grid['COND'] = (modflow_feature_grid['Wid2'] * modflow_feature_grid['Len3']) / modflow_feature_grid['Dep2']
        elif feature_type == "recharge":
            modflow_feature_grid['COND'] = modflow_feature_grid['perc']
            
    else:
        
        print(f'MODFLOW grid was not created yet. Using SWAT model {feature_type} shapefile for initialization of package.')
        modflow_feature_grid = original_feature.copy()
        if feature_type == "lakes":
            modflow_feature_grid['lake_full_area'] = modflow_feature_grid.geometry.area
            modflow_feature_grid['COND'] = modflow_feature_grid['lake_full_area']
        elif feature_type == "rivers":
            modflow_feature_grid['COND'] = (modflow_feature_grid['Wid2'] * modflow_feature_grid['Len2']) / modflow_feature_grid['Dep2']
        elif feature_type == "recharge":
            modflow_feature_grid['COND'] = modflow_feature_grid['perc']
    # Save as a temporary shapefile
    modflow_feature_grid.to_file(temp_feature_path)

    # Set arcpy environment variables
    current_directory = '/data/MyDataBase/SWATGenXAppData/'
    env.workspace = current_directory

    # Use the cell size of the reference raster
    cell_size = arcpy.GetRasterProperties_management(ref_raster_path, "CELLSIZEX").getOutput(0)
    
    env.workspace = '/data/MyDataBase/SWATGenXAppData/'
    arcpy.env.overwriteOutput = True  # Enable overwrite
    arcpy.env.snapRaster = f"/data/MyDataBase/SWATGenXAppData/all_rasters/DEM_250m.tif"  # Clear the snap raster setting
    arcpy.env.cellSize = RESOLUTION  # Setting cell size
    arcpy.env.outputCoordinateSystem = arcpy.Describe(f"/data/MyDataBase/SWATGenXAppData/all_rasters/DEM_250m.tif").spatialReference
    arcpy.env.nodata = "NONE"
    arcpy.env.overwriteOutput = True
    arcpy.env.extent = arcpy.Describe(f"/data/MyDataBase/SWATGenXAppData/all_rasters/DEM_250m.tif").extent

    
    # Convert the temporary shapefile to raster
    if feature_type == "lakes":
        arcpy.PolygonToRaster_conversion(temp_feature_path, "COND", output_raster_path, cellsize=cell_size)
    elif feature_type == "rivers":
        arcpy.PolylineToRaster_conversion(temp_feature_path, "COND", output_raster_path, cellsize=cell_size)
    elif feature_type == "recharge":
        arcpy.PolylineToRaster_conversion(temp_feature_path, "COND", output_raster_path, cellsize=cell_size)
    
    # Delete the temporary shapefile
    arcpy.Delete_management(temp_feature_path)
                                                                
        # Define the output path for the feature class and raster
rasters_output = f'/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/{NAME}/MODFLOW_250/rasters_input/'
ref_raster_path = os.path.join(rasters_output, f'{NAME}_raster.tif')
swat_recharge_raster_path = f'/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12/{NAME}/SWAT_MODEL/SWAT_outputs/{NAME}_recharge.tif'
feature_class_path = os.path.join('/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/huc12','40500010102','SWAT_MODEL/Watershed/Shapes/hrus1.shp')                                           
rasterize_SWAT_features("lakes", swat_recharge_raster_path, ref_raster_path,  load_raster_args)

NameError: name 'lists_of_huc' is not defined

In [ ]:
recharge_hru_temp
